In [3]:
import esm
import torch
import time
import gc
import os
import numpy as np
from Bio import SeqIO
import pandas as pd
from cyvcf2 import VCF

%run "../scripts/data_processing.py"
%run "../scripts/node_edge_generation.py"
%run "../scripts/graph_functions.py"

#Data path
datadir = os.path.abspath("../../Data/")

In [2]:
os.system("ls -ltr "+ datadir+"/GWAS/PRJNA1104541")

total 38792996
-rwxr-xr-x 1 louie louie  134883391 Sep 16 14:44 SRR28808391.fastq.gz
-rwxr-xr-x 1 louie louie  175829790 Sep 16 14:47 SRR28808366.fastq.gz
-rwxr-xr-x 1 louie louie  137556987 Sep 16 14:47 SRR28808369.fastq.gz
-rwxr-xr-x 1 louie louie  125930394 Sep 16 14:47 SRR28808379.fastq.gz
-rwxr-xr-x 1 louie louie  208640818 Sep 16 14:48 SRR28808393.fastq.gz
-rwxr-xr-x 1 louie louie  216681502 Sep 16 14:48 SRR28808373.fastq.gz
-rwxr-xr-x 1 louie louie  251263758 Sep 16 14:48 SRR28808365.fastq.gz
-rwxr-xr-x 1 louie louie  298368113 Sep 16 14:48 SRR28808367.fastq.gz
-rwxr-xr-x 1 louie louie  256624158 Sep 16 14:48 SRR28808372.fastq.gz
-rwxr-xr-x 1 louie louie  270419069 Sep 16 14:48 SRR28808368.fastq.gz
-rwxr-xr-x 1 louie louie  215636846 Sep 16 14:48 SRR28808370.fastq.gz
-rwxr-xr-x 1 louie louie  199309783 Sep 16 14:48 SRR28808376.fastq.gz
-rwxr-xr-x 1 louie louie  157074821 Sep 16 14:51 SRR28808380.fastq.gz
-rwxr-xr-x 1 louie louie  139678662 Sep 16 14:51 SRR28808392.fastq.gz
-rwxr

0

In [2]:
#reference = datadir+"/GWAS/Hsapiens_ref/NCBI_GRCh38.p14/GCF_000001405.40_GRCh38.p14_genomic.fna"
reference = datadir+"/GWAS/Hsapiens_ref/ENSEMBL_GRCh38.p14/Homo_sapiens.GRCh38.dna_sm.primary_assembly.fa"
start = time.time()
# Index the reference genome
os.system(f'bwa index {reference}')
print(f'Reference index complete | {(time.time()-start)/60} minutes')

[bwa_index] Pack FASTA... 9.49 sec
[bwa_index] Construct BWT for the packed sequence...
[BWTIncCreate] textLength=6199501436, availableWord=448219384
[BWTIncConstructFromPacked] 10 iterations done. 99999996 characters processed.
[BWTIncConstructFromPacked] 20 iterations done. 199999996 characters processed.
[BWTIncConstructFromPacked] 30 iterations done. 299999996 characters processed.
[BWTIncConstructFromPacked] 40 iterations done. 399999996 characters processed.
[BWTIncConstructFromPacked] 50 iterations done. 499999996 characters processed.
[BWTIncConstructFromPacked] 60 iterations done. 599999996 characters processed.
[BWTIncConstructFromPacked] 70 iterations done. 699999996 characters processed.
[BWTIncConstructFromPacked] 80 iterations done. 799999996 characters processed.
[BWTIncConstructFromPacked] 90 iterations done. 899999996 characters processed.
[BWTIncConstructFromPacked] 100 iterations done. 999999996 characters processed.
[BWTIncConstructFromPacked] 110 iterations done. 1

Reference index complete | 36.07257577578227 minutes


692.61 sec
[main] Version: 0.7.17-r1188
[main] CMD: bwa index /home/louie/Projects/Personal/GraphDB/Data/GWAS/Hsapiens_ref/ENSEMBL_GRCh38.p14/Homo_sapiens.GRCh38.dna_sm.primary_assembly.fa
[main] Real time: 2164.351 sec; CPU: 2158.586 sec


In [4]:
maindir   = datadir+"/GWAS/PRJNA1104541/"
snpeff    = "/home/louie/Documents/Software/snpEff/snpEff.jar"

count = 0
for i in os.listdir(maindir):
    start = time.time()
    if "fastq" in i and ".gz" not in i:
        count += 1
        stem = i.split(".fastq")[0]
        loc_short = f'{maindir}{stem}/{stem}'
        print(f'{"\n"*2}{"~"*10}\n({count}) - Main file - {i}')
        # Make directory
        if os.path.isdir(f'{maindir}{stem}') == False:
            os.system(f'mkdir {maindir}{stem}')
        os.system(f'cp {maindir}{i} {maindir}{stem}')
        # Align reads to reference (single-end)
        os.system(f'\nbwa mem -t 12 {reference} {loc_short}.fastq > {loc_short}_aligned.sam')
        # Convert SAM to BAM
        os.system(f'\nsamtools view -Sb {loc_short}_aligned.sam | samtools sort -o {loc_short}_aligned.bam')
        # Pileup generation
        os.system(f'\nbcftools mpileup -f {reference} {loc_short}_aligned.bam > {loc_short}.bcf')
        # Call variants
        os.system(f'\nbcftools call -mv -Ov -o {loc_short}.vcf {loc_short}.bcf')
        # Annotate VCF
        os.system(f'\njava -Xmx4g -jar {snpeff} GRCh38.p14 {loc_short}.vcf > {loc_short}_annotated.vcf')
        # Clean intermediary files
        os.system(f'rm {loc_short}.bcf {loc_short}_aligned.bam {loc_short}_aligned.sam {loc_short}.fastq {loc_short}.vcf')
        print(f'{i} analysis complete | {round((time.time()-start)/60, 3)} minutes')



~~~~~~~~~~
(1) - Main file - SRR28808386.fastq


[M::bwa_idx_load_from_disk] read 0 ALT contigs
[M::process] read 1682952 sequences (120000050 bp)...
[M::process] read 1681744 sequences (120000007 bp)...
[M::mem_process_seqs] Processed 1682952 reads in 439.702 CPU sec, 36.785 real sec
[M::process] read 1020803 sequences (72939405 bp)...
[M::mem_process_seqs] Processed 1681744 reads in 452.644 CPU sec, 37.817 real sec
[M::mem_process_seqs] Processed 1020803 reads in 274.654 CPU sec, 22.919 real sec
[main] Version: 0.7.17-r1188
[main] CMD: bwa mem -t 12 /home/louie/Projects/Personal/GraphDB/Data/GWAS/Hsapiens_ref/ENSEMBL_GRCh38.p14/Homo_sapiens.GRCh38.dna_sm.primary_assembly.fa /home/louie/Projects/Personal/GraphDB/Data/GWAS/PRJNA1104541/SRR28808386/SRR28808386.fastq
[main] Real time: 101.531 sec; CPU: 1169.784 sec
[bam_sort_core] merging from 1 files and 1 in-memory blocks...
[mpileup] 1 samples in 1 input files
[mpileup] maximum number of reads per input file set to -d 250
Note: none of --samples-file, --ploidy or --ploidy-file given

SRR28808386.fastq analysis complete | 3.7725327372550965 minutes


~~~~~~~~~~
(2) - Main file - SRR28808390.fastq


[M::bwa_idx_load_from_disk] read 0 ALT contigs
[M::process] read 1656520 sequences (120000008 bp)...
[M::process] read 1655932 sequences (120000057 bp)...
[M::mem_process_seqs] Processed 1656520 reads in 433.221 CPU sec, 36.272 real sec
[M::process] read 830676 sequences (60279008 bp)...
[M::mem_process_seqs] Processed 1655932 reads in 427.793 CPU sec, 35.731 real sec
[M::mem_process_seqs] Processed 830676 reads in 218.493 CPU sec, 18.213 real sec
[main] Version: 0.7.17-r1188
[main] CMD: bwa mem -t 12 /home/louie/Projects/Personal/GraphDB/Data/GWAS/Hsapiens_ref/ENSEMBL_GRCh38.p14/Homo_sapiens.GRCh38.dna_sm.primary_assembly.fa /home/louie/Projects/Personal/GraphDB/Data/GWAS/PRJNA1104541/SRR28808390/SRR28808390.fastq
[main] Real time: 93.063 sec; CPU: 1081.961 sec
[bam_sort_core] merging from 1 files and 1 in-memory blocks...
[mpileup] 1 samples in 1 input files
[mpileup] maximum number of reads per input file set to -d 250
Note: none of --samples-file, --ploidy or --ploidy-file given, a

SRR28808390.fastq analysis complete | 3.1317101319630942 minutes


~~~~~~~~~~
(3) - Main file - SRR28808376.fastq


...

SRR28808375.fastq analysis complete | 7.125429578622183 minutes


~~~~~~~~~~
(29) - Main file - SRR28808382.fastq


[M::bwa_idx_load_from_disk] read 0 ALT contigs
[M::process] read 1674202 sequences (120000012 bp)...
[M::process] read 1672530 sequences (120000088 bp)...
[M::mem_process_seqs] Processed 1674202 reads in 445.400 CPU sec, 37.240 real sec
[M::process] read 1246586 sequences (89519403 bp)...
[M::mem_process_seqs] Processed 1672530 reads in 452.796 CPU sec, 37.830 real sec
[M::mem_process_seqs] Processed 1246586 reads in 338.372 CPU sec, 28.232 real sec
[main] Version: 0.7.17-r1188
[main] CMD: bwa mem -t 12 /home/louie/Projects/Personal/GraphDB/Data/GWAS/Hsapiens_ref/ENSEMBL_GRCh38.p14/Homo_sapiens.GRCh38.dna_sm.primary_assembly.fa /home/louie/Projects/Personal/GraphDB/Data/GWAS/PRJNA1104541/SRR28808382/SRR28808382.fastq
[main] Real time: 106.361 sec; CPU: 1238.949 sec
[bam_sort_core] merging from 1 files and 1 in-memory blocks...
[mpileup] 1 samples in 1 input files
[mpileup] maximum number of reads per input file set to -d 250
Note: none of --samples-file, --ploidy or --ploidy-file given

SRR28808382.fastq analysis complete | 3.6728576461474103 minutes


~~~~~~~~~~
(30) - Main file - SRR28808387.fastq


[M::bwa_idx_load_from_disk] read 0 ALT contigs
[M::process] read 1659840 sequences (120000049 bp)...
[M::process] read 1658732 sequences (120000146 bp)...
[M::mem_process_seqs] Processed 1659840 reads in 488.968 CPU sec, 40.895 real sec
[M::process] read 1459568 sequences (105639900 bp)...
[M::mem_process_seqs] Processed 1658732 reads in 496.526 CPU sec, 41.431 real sec
[M::mem_process_seqs] Processed 1459568 reads in 436.774 CPU sec, 36.482 real sec
[main] Version: 0.7.17-r1188
[main] CMD: bwa mem -t 12 /home/louie/Projects/Personal/GraphDB/Data/GWAS/Hsapiens_ref/ENSEMBL_GRCh38.p14/Homo_sapiens.GRCh38.dna_sm.primary_assembly.fa /home/louie/Projects/Personal/GraphDB/Data/GWAS/PRJNA1104541/SRR28808387/SRR28808387.fastq
[main] Real time: 121.168 sec; CPU: 1424.569 sec
[bam_sort_core] merging from 1 files and 1 in-memory blocks...
[mpileup] 1 samples in 1 input files
[mpileup] maximum number of reads per input file set to -d 250
Note: none of --samples-file, --ploidy or --ploidy-file give

SRR28808387.fastq analysis complete | 4.088012055555979 minutes


In [ ]:
import time
import os
import pandas as pd
from cyvcf2 import VCF

#Data path
datadir = os.path.abspath("../../Data/")

#maindir   = datadir+"/GWAS/PRJNA1104541/"
#snpeff    = "/home/louie/Documents/Software/snpEff/snpEff.jar"

def IndexRefGenome(reference):
    start = time.time()
    # Index the reference genome with BWA
    os.system(f'bwa index {reference}')
    print(f'Reference index complete | {(time.time()-start)/60} minutes')

def GenerateVCF(maindir, snpeff):
    count = 0
    for i in os.listdir(maindir):
        start = time.time()
        if "fastq" in i and ".gz" not in i:
            count += 1
            stem = i.split(".fastq")[0]
            loc_short = f'{maindir}{stem}/{stem}'
            print(f'{"\n"*2}{"~"*10}\n({count}) - Main file - {i}')
            # Make directory
            if os.path.isdir(f'{maindir}{stem}') == False:
                os.system(f'mkdir {maindir}{stem}')
            os.system(f'cp {maindir}{i} {maindir}{stem}')
            # Align reads to reference (single-end)
            os.system(f'\nbwa mem -t 12 {reference} {loc_short}.fastq > {loc_short}_aligned.sam')
            # Convert SAM to BAM
            os.system(f'\nsamtools view -Sb {loc_short}_aligned.sam | samtools sort -o {loc_short}_aligned.bam')
            # Pileup generation
            os.system(f'\nbcftools mpileup -f {reference} {loc_short}_aligned.bam > {loc_short}.bcf')
            # Call variants
            os.system(f'\nbcftools call -mv -Ov -o {loc_short}.vcf {loc_short}.bcf')
            # Annotate VCF
            os.system(f'\njava -Xmx4g -jar {snpeff} GRCh38.p14 {loc_short}.vcf > {loc_short}_annotated.vcf')
            # Clean intermediary files
            os.system(f'rm {loc_short}.bcf {loc_short}_aligned.bam {loc_short}_aligned.sam {loc_short}.fastq {loc_short}.vcf')
            print(f'{i} analysis complete | {round((time.time()-start)/60, 3)} minutes')

def ParseVCF(maindir):
    #Set default df
    df        = pd.DataFrame(columns = ["Chr", "Pos","Ref", "Alt","Gene","Effect","Impact","Transcript","ProteinChange"])

    #Iterate through VCF files in directory, return df with "High" impact varients
    start = time.time()
    for path, subdirs, files in os.walk(maindir):
        for name in files:
            if "annotated.vcf" in name:
                vcf_file = os.path.join(path, name)
                print(vcf_file)
                records = []
                for variant in VCF(vcf_file):
                    ann_field = variant.INFO.get("ANN")
                    for ann in ann_field.split(","):
                        fields = ann.split("|")
                        records.append({
                            "Chr": variant.CHROM,
                            "Pos": variant.POS,
                            "Ref": variant.REF,
                            "Alt": ",".join(variant.ALT),
                            "Gene": fields[3],
                            "Effect": fields[1],
                            "Impact": fields[2],
                            "Transcript": fields[6],
                            "ProteinChange": fields[10]
                        })
                tmpdf = pd.DataFrame(records)
                df = pd.concat([df, tmpdf.loc[tmpdf["Impact"] == "HIGH"]])
                
    print(f'DF generation complete | {round((time.time()-start)/60, 3)} minutes')
    
    return df

In [16]:
maindir   = datadir+"/GWAS/PRJNA1104541/"
df        = pd.DataFrame(columns = ["Chr", "Pos","Ref", "Alt","Gene","Effect","Impact","Transcript","ProteinChange"])

start = time.time()
for path, subdirs, files in os.walk(maindir):
    for name in files:
        if "annotated.vcf" in name:
            vcf_file = os.path.join(path, name)
            print(vcf_file)
            records = []
            for variant in VCF(vcf_file):
                ann_field = variant.INFO.get("ANN")
                for ann in ann_field.split(","):
                    fields = ann.split("|")
                    records.append({
                        "Chr": variant.CHROM,
                        "Pos": variant.POS,
                        "Ref": variant.REF,
                        "Alt": ",".join(variant.ALT),
                        "Gene": fields[3],
                        "Effect": fields[1],
                        "Impact": fields[2],
                        "Transcript": fields[6],
                        "ProteinChange": fields[10]
                    })
            tmpdf = pd.DataFrame(records)
            df = pd.concat([df, tmpdf.loc[tmpdf["Impact"] == "HIGH"]])
print(f'VCF parsing complete | {round((time.time()-start)/60, 3)} minutes')

/home/louie/Projects/Personal/GraphDB/Data/GWAS/PRJNA1104541/SRR28808368/SRR28808368_annotated.vcf
/home/louie/Projects/Personal/GraphDB/Data/GWAS/PRJNA1104541/SRR28808377/SRR28808377_annotated.vcf
/home/louie/Projects/Personal/GraphDB/Data/GWAS/PRJNA1104541/SRR28808385/SRR28808385_annotated.vcf
/home/louie/Projects/Personal/GraphDB/Data/GWAS/PRJNA1104541/SRR28808370/SRR28808370_annotated.vcf
/home/louie/Projects/Personal/GraphDB/Data/GWAS/PRJNA1104541/SRR28808381/SRR28808381_annotated.vcf
/home/louie/Projects/Personal/GraphDB/Data/GWAS/PRJNA1104541/SRR28808392/SRR28808392_annotated.vcf
/home/louie/Projects/Personal/GraphDB/Data/GWAS/PRJNA1104541/SRR28808364/SRR28808364_annotated.vcf
/home/louie/Projects/Personal/GraphDB/Data/GWAS/PRJNA1104541/SRR28808379/SRR28808379_annotated.vcf
/home/louie/Projects/Personal/GraphDB/Data/GWAS/PRJNA1104541/SRR28808388/SRR28808388_annotated.vcf
/home/louie/Projects/Personal/GraphDB/Data/GWAS/PRJNA1104541/SRR28808371/SRR28808371_annotated.vcf
/home/loui

In [32]:
df["Gene"].unique()

array(['SDF4', 'SPEN', 'RPL11', 'TRNAU1AP', 'EPHA10', 'MACF1', 'NRDC',
       'PLPP3', 'SRSF11', 'RPL5', 'DDX20', 'S100A9', 'S100A8', 'KRTCAP2',
       'ASH1L', 'IFI16', 'FMO2', 'CEP350', 'LAMC2', 'LBR', 'GUK1',
       'SDCCAG8', 'AKT3', 'OPTN', 'MLLT10', 'HNRNPH3', 'CCAR1', 'ADIRF',
       'BORCS7-ASMT', 'AS3MT', 'GSTO1', 'LOC124902554', 'SEC23IP',
       'ACADSB', 'IFITM3', 'EPS8L2', 'RPL27A', 'RPS13', 'PEX16', 'PSMC3',
       'NDUFS3', 'SLC15A3', 'WDR74', 'FAU', 'YIF1A', 'RNF169', 'FAT3',
       'CWF19L2', 'NNMT', 'ATP5MG', 'KMT2A', 'KCNJ1', 'WNK1', 'NDUFA9',
       'PTMS', 'A2M', 'PLBD1', 'MYG1', 'BAZ2A', 'CNOT2', 'ATP2B1',
       'UHRF1BP1L', 'HSP90B1', 'SART3', 'ALKBH2', 'CLIP1', 'ZNF891',
       'POMP', 'MRPL52', 'PSME1', 'BAZ1A', 'NEMF', 'KTN1', 'DAAM1',
       'ZBTB25', 'RBM25', 'AREL1', 'SLIRP', 'CALM1', 'IGHGP', 'INO80',
       'DUT', 'ZNF609', 'RPLP1', 'MYO9A', 'UBE2Q2', 'SCAPER', 'PSMA4',
       'AKAP13', 'MCTP2', 'HBA1', 'METTL26', 'TSC2', 'SRRM2', 'ELOB',
       'SMIM22'

In [17]:
for i in df.loc[df["ProteinChange"] != ""]["Effect"].unique():
    print(i)

frameshift_variant
stop_lost
frameshift_variant&stop_lost
frameshift_variant&splice_region_variant
stop_gained
frameshift_variant&stop_gained
start_lost
stop_gained&splice_region_variant
stop_lost&conservative_inframe_deletion
frameshift_variant&splice_acceptor_variant&splice_region_variant&intron_variant
start_lost&disruptive_inframe_deletion
splice_acceptor_variant&conservative_inframe_deletion&splice_region_variant&intron_variant
frameshift_variant&splice_donor_variant&splice_region_variant&intron_variant
frameshift_variant&start_lost
frameshift_variant&stop_gained&splice_region_variant
frameshift_variant&stop_lost&splice_region_variant
start_lost&splice_region_variant
stop_lost&disruptive_inframe_deletion
splice_donor_variant&conservative_inframe_deletion&splice_region_variant&intron_variant


In [18]:
display(df.loc[df["ProteinChange"] != ""])

,Chr,Pos,Ref,Alt,Gene,Effect,Impact,Transcript,ProteinChange
626,1,1652532,GT,G,CDK11B,frameshift_variant,HIGH,XM_011542490.4,p.Gln89fs
627,1,1652532,GT,G,CDK11B,frameshift_variant,HIGH,XM_011542494.4,p.Gln55fs
628,1,1652532,GT,G,CDK11B,frameshift_variant,HIGH,XM_017002928.3,p.Gln55fs
629,1,1652532,GT,G,CDK11B,frameshift_variant,HIGH,XM_047435355.1,p.Gln55fs
630,1,1652532,GT,G,CDK11B,frameshift_variant,HIGH,XM_047435360.1,p.Gln55fs
...,...,...,...,...,...,...,...,...,...
2727828,X,149490351,CA,CAA,IDS,frameshift_variant,HIGH,NM_000202.8,p.Ala324fs
2727829,X,149490351,CA,CAA,IDS,frameshift_variant,HIGH,NM_001166550.4,p.Ala234fs
2727830,X,149490351,CA,CAA,IDS,frameshift_variant,HIGH,NM_006123.5,p.Ala324fs
2729534,X,153935301,G,T,RENBP,stop_gained,HIGH,NM_002910.6,p.Cys423*


In [12]:
len(df.loc[df["ProteinChange"] != ""]["ProteinChange"].unique())

18660

In [16]:
import pandas as pd
from cyvcf2 import VCF

testvcf = "/home/louie/Projects/Personal/GraphDB/Data/GWAS/PRJNA1104541/SRR28808386/SRR28808386_annotated.vcf"

records = []
for variant in VCF(testvcf):
    ann_field = variant.INFO.get("ANN")
    #if not ann_field:
    #    continue
    for ann in ann_field.split(","):
        fields = ann.split("|")
        records.append({
            "CHROM": variant.CHROM,
            "POS": variant.POS,
            "REF": variant.REF,
            "ALT": ",".join(variant.ALT),
            "Gene": fields[3],
            "Effect": fields[1],
            "Impact": fields[2],
            "Transcript": fields[6],
            "ProteinChange": fields[10]
        })

df = pd.DataFrame(records)
display(df.head(100))



,CHROM,POS,REF,ALT,Gene,Effect,Impact,Transcript,ProteinChange
0,1,630665,T,C,LOC101928626,upstream_gene_variant,MODIFIER,NR_125957.1,
1,1,630665,T,C,MIR12136,downstream_gene_variant,MODIFIER,NR_162149.1,
2,1,630665,T,C,MIR12136,downstream_gene_variant,MODIFIER,unassigned_transcript_4,
3,1,630665,T,C,MTND2P28,intragenic_variant,MODIFIER,MTND2P28,
4,1,630668,G,A,LOC101928626,upstream_gene_variant,MODIFIER,NR_125957.1,
...,...,...,...,...,...,...,...,...,...
95,1,1218635,T,C,TNFRSF4,upstream_gene_variant,MODIFIER,XM_011542076.3,
96,1,1218635,T,C,TNFRSF4,upstream_gene_variant,MODIFIER,NM_003327.4,
97,1,1218635,T,C,TNFRSF4,upstream_gene_variant,MODIFIER,XM_011542075.3,
98,1,1218635,T,C,TNFRSF4,upstream_gene_variant,MODIFIER,XR_007063145.1,
